In [ ]:
# training a xgboost model to predix suffix of the processes
import pandas as pd
import numpy as np
import xgboost as xgb
from aux_functions import split_data
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('data/preprocessed/BPI_Challenge_2012.csv')

In [ ]:
dropping_clomns = ['time:timestamp', 'case:REG_DATE', 'next_timestamp']
df.drop(dropping_clomns, axis=1, inplace=True)
df

In [ ]:
# frequency of each length of the traces in a dictionary
trace_lengths = df['case:concept:name'].value_counts().to_dict()
trace_lengths_frequency = {}
for process, key in trace_lengths.items():
    if key in trace_lengths_frequency:
        trace_lengths_frequency[key] += 1
    else:
        trace_lengths_frequency[key] = 1

trace_lengths_frequency

In [ ]:
# sort trace_lengths_frequency by key
trace_lengths_frequency = dict(sorted(trace_lengths_frequency.items()))
# Getting the cumulative sum of the values
cumulative_sum = 0
cum_tace_lengths_frequency = {}
for key, value in trace_lengths_frequency.items():
    cumulative_sum += value
    cum_tace_lengths_frequency[key] = cumulative_sum

cum_tace_lengths_frequency

In [ ]:
# Getting the percentage of the cumulative sum
total = sum(trace_lengths_frequency.values())
cum_tace_lengths_frequency_percentage = {}
for key, value in cum_tace_lengths_frequency.items():
    cum_tace_lengths_frequency_percentage[key] = value/total

cum_tace_lengths_frequency_percentage

In [ ]:
# Due to do this we cut of the traces with length more than 50 since they are only 8% of the data
df = df[df['case:concept:name'].isin(df.groupby('case:concept:name').size()[df.groupby('case:concept:name').size() <= 50].index)]

In [ ]:
train, test = split_data(df, 0.8) 

In [ ]:
concept_name_train_copy = train['concept:name'].copy()
concept_name_test_copy = test['concept:name'].copy()

# one hot encoding on the concept:name and lifecycle:transition
train = pd.get_dummies(train, columns=['concept:name', 'lifecycle:transition'], )
test = pd.get_dummies(test, columns=['concept:name', 'lifecycle:transition'])
train['concept:name'] = concept_name_train_copy
test['concept:name'] = concept_name_test_copy
train

In [ ]:
prediction_columns = ['case:concept:name', 'concept:name', 'position']

In [ ]:
# Generating all the possible prefixes and suffixes of length 5 in the train set
prefix = []
suffix = []
for case in train['case:concept:name'].unique():
    case_df = train[train['case:concept:name'] == case]
    for i in range(5, len(case_df)):
        prefix.append(case_df.iloc[i-5:i][prediction_columns])
        suffix.append(case_df.iloc[i:][prediction_columns])

# DO NOT RUN

In [ ]:
# Defining what category each column fall into
dropping_clomns = ['case:concept:name', 'next_concept:name']
train.drop(dropping_clomns, axis=1, inplace=True)

In [ ]:
# Dropping the 'case:concept:name' column and next_concept:name column from the train and test suffixes
# Which are the 2nd and 4th columns
prefixes = [np.delete(prefix, [1,4], axis=1) for prefix in prefixes]
# prefixes_test = [np.delete(prefix, [1,4], axis=1) for prefix in prefixes_test]

In [ ]:
train_dmatrix = xgb.DMatrix(data=prefixes[0], label=np.array(suffixes[0]))